In [18]:
import pandas as pd
import pickle
from model import DATASET_PATH

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
file_name = f"data/dump/{DATASET_PATH}/BERT_data_for_classifier/GCN_turn3_predictedTest.pkl"
file_name = f"data/dump/{DATASET_PATH}/BERT_data_for_classifier/EGAT_turn10_predictedTest.pkl"
file_name = f"data/dump/{DATASET_PATH}/BERT_data_for_classifier/bert_predictedTest.pkl"

# df_predictions = pd.DataFrame({
#     'true_label': all_labels,
#     'predicted_label': all_preds
# })

In [24]:
X_test.shape

(3400, 12)

In [33]:

with open(model_files["BERT Prediction"], 'rb') as file:
    df_predictions = pickle.load(file)

In [34]:
df_predictions

,true_label,predicted_label
0,6,4
1,0,4
2,4,4
3,4,4
4,4,4
...,...,...
3395,4,4
3396,4,4
3397,0,4
3398,4,4


In [35]:
model_files = {
    "GCN Prediction": f"data/dump/{DATASET_PATH}/BERT_data_for_classifier/GCN_turn3_predictedTest.pkl",
    "EGAT Prediction": f"data/dump/{DATASET_PATH}/BERT_data_for_classifier/EGAT_turn10_predictedTest.pkl",
    "BERT Prediction": f"data/dump/{DATASET_PATH}/BERT_data_for_classifier/bert_predictedTest.pkl"
}

# Load label decoder
with open(f'data/dump/{DATASET_PATH}/label_decoder.pkl', 'rb') as file:
    label_decoder = pickle.load(file)

# Load X_test
X_test = pd.read_csv(f'data/{DATASET_PATH}/test_sent_emo_dya.csv', encoding='utf-8')

# Extract necessary features from X_test
X_test_filtered = X_test[['Utterance', 'Emotion', 'Dialogue_ID', 'Utterance_ID']].rename(columns={'Emotion': 'Actual Emotion'})

# Initialize an empty DataFrame to store predictions
predictions_df = pd.DataFrame(index=X_test_filtered.index)

# Process each model file and add predictions to predictions_df
for column_name, file_path in model_files.items():
    with open(file_path, 'rb') as file:
        df_predictions = pickle.load(file)
    
    # Decode predicted labels
    df_predictions['predicted_label'] = df_predictions['predicted_label'].map(label_decoder)
    
    # Add predictions to predictions_df with the appropriate column name
    predictions_df[column_name] = df_predictions['predicted_label']

# Concatenate X_test_filtered with the predictions_df
final_df = pd.concat([X_test_filtered, predictions_df], axis=1)

# Rearrange the columns to insert predictions between 'Actual Emotion' and 'Dialogue_ID'
cols = list(final_df.columns)
emotion_index = cols.index('Actual Emotion')
dialogue_id_index = cols.index('Dialogue_ID')

# Insert predictions columns between 'Actual Emotion' and 'Dialogue_ID'
for column_name in model_files.keys():
    cols.insert(dialogue_id_index, cols.pop(cols.index(column_name)))

final_df = final_df[cols]

# Save to CSV
output_file = f'data/dump/{DATASET_PATH}/final_predictions.csv'
final_df.to_csv(output_file, index=False, encoding='utf-8')

print(f"Final predictions saved to {output_file}")

Final predictions saved to data/dump/dataset_original/final_predictions.csv


In [36]:
final_df

,Utterance,Actual Emotion,BERT Prediction,EGAT Prediction,GCN Prediction,Dialogue_ID,Utterance_ID
0,Why do all you’re coffee mugs have numbers on ...,surprise,neutral,surprise,neutral,0,0
1,Oh. That’s so Monica can keep track. That way ...,anger,neutral,neutral,neutral,0,1
2,Y'know what?,neutral,neutral,neutral,neutral,0,2
3,Okay.,neutral,neutral,neutral,neutral,1,0
4,"Ross, didn't you say that there was an elevato...",neutral,neutral,surprise,surprise,1,1
...,...,...,...,...,...,...,...
3395,You just wanna each take one?,neutral,neutral,neutral,neutral,576,1
3396,Why don’t you phase it out?,neutral,neutral,anger,neutral,576,2
3397,"Yeah, fade the accent out and people will thin...",anger,neutral,anger,neutral,576,3
3398,Monica you really think I should try this phas...,neutral,neutral,sadness,sadness,576,4
